In [1]:
## Load Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

import seaborn as sns
from sklearn.cluster import KMeans

In [20]:
## Concatenating PBP scoring files
pbp_sc_files = []
for i in np.arange(1,22):
    name = 'PBP_Files/pbpstats_export-{}.csv'.format(i)
    file = pd.read_csv(name)
    file['Season'] = 2022-i
    pbp_sc_files.append(file)
pbp_sc = pd.concat(pbp_sc_files)

## Final pbp scoring csv
pbp_sc.to_csv('pbp_sc.csv')
#pbp_sc.head(2)

In [22]:
## Concatenating PBP assist files
pbp_as_files = []
for i in np.arange(-42,-21):
    name = 'PBP_Files/pbpstats_export-{}.csv'.format(-i)
    file = pd.read_csv(name)
    file['Season'] = 2000-21-i
    pbp_as_files.append(file)    
pbp_as = pd.concat(pbp_as_files)

## Final pbp assist csv
pbp_as.to_csv('pbp_as.csv')
#pbp_as.head(2)

In [23]:
## New PBP Files
# pbp_sc and pbp_as